# DataFrame

- Objekt o dvou dimenzích  (můžeme o něm přemýšlet jako o tabulce v MS Excel)
- Položky DataFrame mohou být jakéhokoliv datového typu (jednotlivé položky mohou být různého datového typu)
- Jednotlivé položky mají index a sloupec, pomocí kterého lze na položky přistupovat
- O DataFrame můžeme přemýšlet také jako o množině Series
- U celé řady operací je nutné definovat, jestli se jedná o operaci napříč indexi / štítky, tj. axis = 0, nebo
operaci nad sloupci tj. axis = 1

## Vytvoření DataFrame

- Za pomoci metody DataFrame(), kdy argumentem je jednodimenzionální či dvoudimenzionální objekt

In [ ]:
# Import potrebnych knihoven
import pandas as pd
# Stazeni .csv souboru publikovaneho na github.com
url = 'https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv'
df = pd.read_csv(url,delimiter = ';')
df['DATE'] = pd.to_datetime(df['DATE'])



In [ ]:
df.head()

### Výběr hodnot na základě pozice, štítku či rozsahu hodnot
- Protože DataFrame je 2D objekt, je třeba definovat index i sloupec položky
- K přistupování na hodnoty slouží metody loc a iloc, jejichž logika je stejná jako u Series

In [ ]:
# Vyber hodnot na indexu 0 az 2
print(df.iloc[0:2])

# Vyber hodnot na indexu 0 az 2 a ve sloupcich 1 a dalsi
print(df.iloc[0:2,1:])

# Vyber hodnot na vsech indexech, sloupcich 2 a dale
print(df.iloc[:,1:])



In [ ]:
# Vyber hodnot na stitku 0 az 2
print(df.loc[0:2])

# Vyber hodnot na stitku 0 az 2 a ve sloupcich NAME  dalsich
print(df.loc[0:2,'NAME':])


# Vyber hodnot na vsech stitcich a ve sloupcich NAME  dalsich
print(df.loc[:,'NAME':])


## Iterace DataFrame

- DataFrame lze iterovat za pomoci metod:
    - iterrows() iterujícími položkami dataframe po řádcích
    - items() iterujícími položkami dataframe po sloupcích
    - itertuples() iterujícími řádky s návratovou hodnotou tuple
    


In [ ]:
# Iterace po sloupcich
for i,r in df.items():
    print(i)
    if i == 'PRCP':
        break

# Iterace po radcich
for i,r in df.iterrows():
    print(i)
    if i == 3:
        break
        



# Iterace po radcich s navratovou hodnotou tuple
for i in df[['NAME','PRCP']].itertuples():
    print(i)
    if i == 3:
        break


In [ ]:
df2 = df.copy()

for i,r in df2.iterrows():
    stanice = r['STATION']
    print(stanice)
    break

# Práce se sloupci a řádky

- U DataFrame můžeme aktualizovat stávající hodnoty sloupce, přidat nový sloupec či odstranit sloupec
    - Odebrání sloupce pomocí metody drop(), viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html
    - Přidání sloupce:
        - Pomocí metody assign(), viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.assign.html
        - Pojmenováním sloupce a definicí hodnoty

In [ ]:
# Vytvoreni kopie DataFrame za pomoci metody copy(), viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.copy.html
srazky = df.copy()


# Odstranujeme hodnoty na urovni radku, tj. axis = 0
srazky.drop([1,2,3],axis=0,inplace=True)
# Odstranujeme hodnoty na urovni sloupcu, tj. axis = 1
srazky.drop(['STATION','TAVG'],axis = 1,inplace=True)


# Pridani novych sloupcu do DataFrame
srazky = srazky.assign(ROK=srazky['DATtE'].d.year)
srazky['MESIC'] = srazky['DATE'].dt.month
srazky.head()



# Úprava dat  funkcemi

- V rámci iterace za pomoci metody at()
- Za pomoci metody apply(), která jako argument přijímá funkci, kterou volá, viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html
- Za pomoci metody map(), která jako argument přijímá funkci, kterou volá, viz https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.map.html
- Hodnoty lze přidat či upravovat i za pomoci vlastních funkcí, ať už pojmenovaných či nejpojmenovaných (lambda funkce)

- Priority při úpravě dat DataFrame
    - použití tzv. vektorizace
    - použití metody apply()
    - iterace za pomoci iterrows(), iteritems(), itertuples()
    


In [20]:

srazky = df.copy()
# Pridani sloupce tzv. vektorizovanym zpusobem
srazky['NAME_L1'] = srazky['NAME'].str.lower()
# Pridani sloupce za pomoci metody apply()
srazky['NAME_L2'] = srazky['NAME'].apply(str.lower)
srazky['NAME_L3'] = None
for i,r in srazky.iterrows():
    srazky.at[i,'NAME_L3'] = r['NAME'].lower()



print(srazky.filter(like='NAME',axis=1).head())


     NAME NAME_L1 NAME_L2 NAME_L3
0  PIDING  piding  piding  piding
1  PIDING  piding  piding  piding
2  PIDING  piding  piding  piding
3  PIDING  piding  piding  piding
4  PIDING  piding  piding  piding


In [ ]:
srazky_vlastni = srazky.copy()

def typ_srazek(srazky):
    if pd.isna(srazky):
        return 'nezjisteno'
    elif srazky == 0:
        return 'neprselo'
    elif srazky > 0 and srazky < 5:
        return 'prselo malo'
    elif srazky >= 5 and srazky < 10:
        return 'prselo'
    else:
        return 'prselo moc'

    

srazky_vlastni['TYP_SRAZEK'] = srazky_vlastni['PRCP'].apply(typ_srazek)


srazky_vlastni['TYP_SRAZEK2'] = None

for i,r in srazky_vlastni.iterrows():

    srazky.at[i,'TYP_SRAZEK2'] = typ_srazek(r['PRCP'])
    if i > 100:
        break
print(srazky.head(20))

### Porovnání rychlosti jednotlivých způsobů přidání nového sloupce

In [ ]:
import time
import numpy as np

srazky1 = srazky.copy()

start = time.time()
srazky1['PRSELO'] = None
for i,r in srazky1.iterrows():
    if pd.isna(r['PRCP']):
        srazky1.at[i,'PRSELO'] = 'nezjisteno'
    elif r['PRCP'] == 0:
        srazky1.at[i,'PRSELO'] = 'ano'
    else:
        srazky1.at[i,'PRSELO'] = 'ne'
end = time.time()
print('iterace: ',end - start)


srazky2 = srazky.copy()
start = time.time()

srazky2['PRSELO'] = srazky2['PRCP'].apply(lambda x: 'nezjisteno' if pd.isna(x) else ('ano' if x > 0 else 'ne'))
end = time.time()
print('apply: ',end-start)

srazky3 = srazky.copy()
start = time.time()

srazky3.loc[pd.isna(srazky3['PRCP']),'PRSELO'] = 'nezjisteno'
srazky3.loc[srazky3['PRCP']==0,'PRSELO'] = 'neprselo'
srazky3.loc[srazky3['PRCP']>0,'PRSELO'] = 'prselo'
end = time.time()
print('loc: ',end - start)


srazky3.head(20)



In [18]:
pd.isna(srazky3['PRCP'])

0        False
1        False
2        False
3        False
4        False
         ...  
67829    False
67830    False
67831    False
67832    False
67833    False
Name: PRCP, Length: 67834, dtype: bool

## Cvičení

### Cvičení zadání
1. Z dat dostupných pod touto URL vytvořte DataFrame s názvem df: 'https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv'
2. DataFrame df omezte na sloupce DATE,NAME,PRCP,TMAX,TMIN
3. Do nového sloupce s názvem ROZDIL_TEPLOT spočítejte rozdíl mezi hodnotami TMAX a TMIN
4. Změňte datový typ sloupce DATE na datetime
5. Do nového sloupce ROK spočítejte rok ze sloupce DATE

6. Vytvořte si kopii DataFrame df, tu pojmenujte df1
7. Zafiltrujte DataFrame df1 tak, aby obsahoval pouze řádky, kde hodnota ve sloupci NAME se rovná 'RUZYNE'
8. Do sloupce ROZDIL_OD_MAX_TEPLOTY spočítejte, o kolik se liší hodnota ve sloupci TMAX na daném řádku od maximální hodnoty ze sloupce TMAX napříč celým DataFrame df1

BONUS:

9. U DataFrame df1 nastavte hodnoty ze sloupce DATE jako index, původní sloupec DATE smažte
10. Zjistěte,jestli jsou hodnoty indexu DataFrame df1 unikátní (tj. neobsahují duplicity)


In [48]:
import pandas as pd
# Stazeni .csv souboru publikovaneho na github.com
url = 'https://raw.githubusercontent.com/petrrozkosny/pydata/main/pydata_data.csv'
df = pd.read_csv(url,delimiter = ';')
df = df[['DATE','NAME','PRCP','TMAX','TMIN']]
df['DATE'] = pd.to_datetime(df['DATE'])
df['ROK'] = df['DATE'].dt.year

df1 = df.copy()
df1 = df1.loc[df1['NAME']=='RUZYNE']
max_t = df1['TMAX'].max()
df1['ROZDIL_OD_MAX_TEPLOTY'] = max_t - df1['TMAX']
df1.index = df1['DATE']
df1 = df1.drop('DATE',axis=1)

# df1.set_index('DATE',drop=True,inplace=True)

print(df1.index.is_unique)


True
407.004


In [44]:
df['MESIC'] = df['DATE'].dt.month
df['POLOLETI'] = df['MESIC'].apply(lambda x: '1. pololeti' if x < 7 else '2. pololeti')
df['ROK'] = df['DATE'].astype(str).str[:4]

bool_vektor = df['NAME'] == 'RUZYNE'
df_ruzyne = df.loc[bool_vektor,'NAME':]
df_ruzyne = df.loc[df['ROK']=='2010',['DATE','PRCP']]
df_ruzyne['PRCP kumulativne'] = df_ruzyne['PRCP'].cumsum()

df_ruzyne.plot.bar(x='DATE',y='PRCP kumulativne',title='Kumulativni srazky na letisti Ruzyne v roce 2010')

<Axes: title={'center': 'Kumulativni srazky na letisti Ruzyne v roce 2010'}, xlabel='DATE'>